# Benchmarking a quantum Reverse Polish Notation Calculator using ZX-Calculus

# Category:
## Innovation:
### Open Project:
#### Deliverable: 
- A Jupyter Notebook containing your documentation, code, and results
discussion. We will need all the files in a zip file if there are any in addition to the
notebook. Alternatively, you may submit a link to a public Github repository
containing all the files.

## Overview:

You can understand this submission as a combination of three topics:

- **Integer Arithmetic using Pennylane**
    - can be either [QFT|https://arxiv.org/pdf/1411.5949.pdf]-[based|https://pennylane.ai/qml/demos/tutorial_qft_arithmetics/] or [not|https://webthesis.biblio.polito.it/15853/1/tesi.pdf]
- **[ZX|https://pennylane.ai/qml/demos/tutorial_zx_calculus/] and [ZXH-Calculus|https://github.com/Quantomatic/pyzx]**
- Circuit optimization, which uses both topics menti

In [2]:
import sys

import matplotlib.pyplot as plt
import pyzx as zx
import pennylane as qml
from pennylane import numpy as np

from collections import deque
from copy import deepcopy
from collections.abc import Generator
from collections.abc import Iterable

In [3]:
"""Helper functions."""

def get_bit_field(k: int) -> Generator:
    for digit in bin(k)[2:]:
        yield 1 if digit=='1' else 0


def pad_zero(bit_field, pad_size: int) -> Generator:
    if pad_size > 0:
        for _ in range(pad_size):
            yield 0
    for bit in bit_field:
        yield bit


def pre_process(a, b) -> tuple[list[int], list[int]]:
    if isinstance(a, np.tensor):
        a = a.tolist()
    if isinstance(b, np.tensor):
        b = b.tolist()
    if isinstance(a, int):
        a = list(get_bit_field(a))
    if isinstance(b, int):
        b = list(get_bit_field(b))
    if hasattr(a, '__len__') and hasattr(b, '__len__') and len(a) != len(b):
        a = list(pad_zero(a, len(b)-len(a)))
        b = list(pad_zero(b, len(a)-len(b)))
    return a, b


def did_padding_work(a, b):
    if hasattr(a, '__len__') and hasattr(b, '__len__'):
        assert len(a) == len(b)
    return len(a)

def from_bit_field(bit_array):
    if isinstance(bit_array, Iterable):
        value = 0
        for bit in bit_array:
            value = (value << 1) | bit
        return value
    return bit_array


# Arithmetic using QFT
## https://pennylane.ai/qml/demos/tutorial_qft_arithmetics/
- Thanks @Guillermo Alonso!
- Tried another approach based on https://webthesis.biblio.polito.it/15853/1/tesi.pdf -- didn't work

In [4]:
def load_to_wires(value, control, wires):
    for j, wire in enumerate(wires):
        qml.CRZ(value * np.pi / (2**j), wires=[control, wire])


def add_(a_wires, b_wires, qft_wires):
    qml.QFT(qft_wires)

    # add a to the counter
    for i, a_wire in enumerate(a_wires):
        load_to_wires(2**(len(a_wires) - i - 1), a_wire, qft_wires)

    # add b to the counter
    for i, b_wire in enumerate(b_wires):
        load_to_wires(2**(len(b_wires) - i - 1), b_wire, qft_wires)

    # return to computational basis
    qml.adjoint(qml.QFT)(wires=qft_wires)


def add(a: int|list, b: int|list) -> (np.tensor, dict):
    a, b = pre_process(a, b)
    n = did_padding_work(a, b)
    num_wires = 3*n + 1

    dev = qml.device("default.qubit", wires=num_wires, shots=1)
    @qml.qnode(dev)
    def circuit():
        total_wires=list(range(num_wires))
        a_wires = total_wires[:n]
        b_wires = total_wires[n:2*n]
        qft_wires = total_wires[2*n:]

        qml.BasisEmbedding(features=a, wires=a_wires)
        qml.BasisEmbedding(features=b, wires=b_wires)
        
        add_(a_wires, b_wires, qft_wires)

        return qml.sample(wires=qft_wires)

    return circuit


def subtract_(a_wires, b_wires, qft_wires):
    qml.broadcast(qml.PauliX, pattern="single", wires=a_wires)
    add_(a_wires, b_wires, qft_wires)
    qml.broadcast(qml.PauliX, pattern="single", wires=qft_wires)


def subtract(a: int|list, b: int|list) -> (np.tensor, dict):
    if from_bit_field(b) > from_bit_field(a):
        raise NotImplementedError("Arithmetic operations for negative integers are not supported yet")

    a, b = pre_process(a, b)
    n = did_padding_work(a, b)
    num_wires = 3*n+1

    dev = qml.device("default.qubit", wires=num_wires, shots=1)
    @qml.qnode(dev)
    def circuit():
        total_wires=list(range(num_wires))
        a_wires = total_wires[:n]
        b_wires = total_wires[n:2*n]
        qft_wires = total_wires[2*n:]

        qml.BasisEmbedding(features=a, wires=a_wires)
        qml.BasisEmbedding(features=b, wires=b_wires)

        subtract_(a_wires, b_wires, qft_wires)

        return qml.sample(wires=qft_wires[1:])


    
    return circuit


def multiply_(a_wires, b_wires, qft_wires):
    qml.QFT(qft_wires)
    for i, a_wire in enumerate(a_wires):
        for j, b_wire in enumerate(b_wires):
            qml.ctrl(load_to_wires, control=a_wire)(
                2 ** (len(a_wires) + len(b_wires) - i - j - 2), control=b_wire, wires=qft_wires)
    # return to computational basis
    qml.adjoint(qml.QFT)(wires=qft_wires)


def multiply(a: int|list, b: int|list) -> (np.tensor, dict):
    a, b = pre_process(a, b)
    n = did_padding_work(a, b)
    num_wires = 4*n

    dev = qml.device("default.qubit", wires=num_wires, shots=1)
    @qml.qnode(dev)
    def circuit():
        total_wires=list(range(num_wires))
        a_wires = total_wires[:n]
        b_wires = total_wires[n:2*n]
        qft_wires = total_wires[2*n:]

        qml.BasisEmbedding(features=a, wires=a_wires)
        qml.BasisEmbedding(features=b, wires=b_wires)
        
        multiply_(a_wires, b_wires, qft_wires)

        return qml.sample(wires=qft_wires)

    return circuit


def divide(a: int|list, b: int|list) -> (np.tensor, dict):
    """
    FIXME:
    Out-of-memory issue with implementation using SWAP and mid-circuit measurement is present.
    Workaround: divide it using classical computing, and later estimate the complexity by
    understanding a // b as a combination of incrementing counter and repeated subtraction
    """
    #a, b = pre_process(a, b)
    #n = did_padding_work(a, b)

    a_val = from_bit_field(a)
    b_val = from_bit_field(b)
    """
    q = min(len(a), len(b))
    num_wires = 3*n+1+q
    dev = qml.device("default.qubit", wires=num_wires, shots=1)
    @qml.qnode(dev)
    def circuit():
        # FIXME: not a real method
        total_wires=list(range(num_wires))
        a_wires = total_wires[:n]
        b_wires = total_wires[n:2*n]
        qft_wires = total_wires[2*n: 3*n+1]
        counter_wires = total_wires[3*n+1:]


        qml.BasisEmbedding(features=a, wires=a_wires)
        qml.BasisEmbedding(features=b, wires=b_wires)
        qml.BasisEmbedding(features=a_val // b_val, wires=counter_wires)
        return qml.sample(wires=counter_wires)
    """

    return lambda : a_val // b_val

In [5]:
class QuantumRPNCalculator:
    """
    A quantum RPN Calculator for unsigned integers.

    It parses the expression and tries different circuit optimization methods:
    - no optimization
    - pennylane's simplication
    - ZX-Calculus.
    """

    def __init__(self, callback=None):
        """Constructor."""

        self.stack = []
        self.naive_q = deque()
        self.pennylane_q = deque()
        self.zx_q = deque()
        self.operators = {
            '+': self._add,
            '-': self._subtract,
            '*': self._multiply,
            '/': self._divide
        }
    def _append_benchmarks(self, circuit):
        """
        Create and append benchmarks.
        When it optimizes a quantum circuit using pyzx.
        It tries to do a full reduce.
        Inspired from https://pennylane.ai/qml/demos/tutorial_zx_calculus/.
        Thanks @Romain Moyard!
        """
        zx_diagram = qml.transforms.to_zx(deepcopy(circuit))()
        self.naive_q.append((qml.specs(circuit)(), zx_diagram))

        simplified_circuit = qml.simplify(deepcopy(circuit))
        zx_diagram_for_simplified_circuit = qml.transforms.to_zx(simplified_circuit)()
        self.pennylane_q.append((qml.specs(simplified_circuit)(), zx_diagram_for_simplified_circuit))

        try:
            zx_diagram_to_reduce = qml.transforms.to_zx(deepcopy(circuit))()
            zx.simplify.full_reduce(zx_diagram_to_full_reduce)
            reduced = pyzx.extract_circuit(zx_diagram_to_full_reduce)
        except:
            zx_diagram_to_reduce = qml.transforms.to_zx(deepcopy(circuit))()
            reduced = zx.simplify.teleport_reduce(zx_diagram_to_reduce)
        self.zx_q.append((reduced.stats(), zx_diagram_to_reduce))
    
    def _get_operands(self):
        """Pop two operands from the stack and return them."""
        return self.stack.pop(), self.stack.pop()
    
    def _add(self):
        rhs, lhs = self._get_operands()
        print(f'{from_bit_field(lhs)}+{from_bit_field(rhs)}')

        circuit = add(lhs, rhs)
        self._append_benchmarks(circuit)
        self.stack.append(circuit())

    def _subtract(self):
        rhs, lhs = self._get_operands()
        print(f'{from_bit_field(lhs)}-{from_bit_field(rhs)}')

        circuit = subtract(lhs, rhs)
        self._append_benchmarks(circuit)
        self.stack.append(circuit())

    def _multiply(self):
        rhs, lhs = self._get_operands()
        print(f'{from_bit_field(lhs)}*{from_bit_field(rhs)}')
        circuit = multiply(lhs, rhs)
        
        self._append_benchmarks(circuit)
        self.stack.append(circuit())

    def _get_result(self):
        return f'RPN calculator result={from_bit_field(self.stack[0])}'
    
    def _divide(self):
        rhs, lhs = self._get_operands()
        lhs = from_bit_field(lhs)
        rhs = from_bit_field(rhs)
        print(f'{lhs}/{rhs}')
        circuit = divide(lhs, rhs)
        
        #self._append_benchmarks(circuit)
        
        self.stack.append(circuit())

    def _show_benchmarks(self):
        for naive_entry, pennylane_entry, zx_entry in zip(self.naive_q, self.pennylane_q, self.zx_q):
            naive_specs, naive_fig = naive_entry
            pennylane_specs, pennylane_fig = pennylane_entry
            zx_specs, zx_fig = zx_entry

            #fig = pyzx.draw_matplotlib(naive_zx_diagram)

            # The following lines are added because the figure is automatically closed by PyZX.
            #manager = plt.figure().canvas.manager
            #manager.canvas.figure = fig
            
            #fig.set_canvas(manager.canvas)

            #plt.show()
            print("=======================================")
            print(f'naive_specs:{naive_specs['resources']}')
            print(f'pennylane_specs:{pennylane_specs['resources']}')
            print(f'zx_specs:{zx_specs}')
            print("=======================================")


    
    def run(self):
        tokens = input('RPN expression:').strip().split()
        for token in tokens:
            if token in self.operators:
            # Operate with the top two numbers on the stack
                self.operators[token]()
            else:
            # We encountered a number: push it onto the top of the stack
                self.stack.append(int(token))
        # Output the answer
        print(self._get_result())
        # Output the benchmark
        self._show_benchmarks()
        

In [7]:
calculator = QuantumRPNCalculator()

In [9]:
calculator.run()

RPN expression: 3 4 *


3*4


/home/dan-seol/micromamba/envs/pennylane-codebook/lib/python3.12/site-packages/pennylane/gradients/gradient_transform.py:293: UserWarning: Attempted to compute the gradient of a tape with no trainable parameters. If this is unintended, please mark trainable parameters in accordance with the chosen auto differentiation framework, or via the 'tape.trainable_params' property.
  warnings.warn(_no_trainable_grad_warning)


RPN calculator result=12
naive_specs:wires: 12
gates: 58
depth: 46
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'C(CRZ)': 54, 'Adjoint(QFT)': 1}
gate_sizes:
{3: 56, 6: 2}
pennylane_specs:wires: 12
gates: 58
depth: 46
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'C(CRZ)': 54, 'Adjoint(QFT)': 1}
gate_sizes:
{3: 56, 6: 2}
zx_specs:Graph(1689 vertices, 2187 edges)
degree distribution: 
1: 24
2: 645
3: 1020



In [10]:

calculator.run()

RPN expression: 1 2 + 3 *


1+2
3*3
RPN calculator result=12
naive_specs:wires: 12
gates: 58
depth: 46
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'C(CRZ)': 54, 'Adjoint(QFT)': 1}
gate_sizes:
{3: 56, 6: 2}
pennylane_specs:wires: 12
gates: 58
depth: 46
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'C(CRZ)': 54, 'Adjoint(QFT)': 1}
gate_sizes:
{3: 56, 6: 2}
zx_specs:Graph(1689 vertices, 2187 edges)
degree distribution: 
1: 24
2: 645
3: 1020

naive_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
pennylane_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
zx_specs:Graph(148 vertices, 183 edges)
degree distribution: 
1: 14
2: 50
3: 84

naive_specs:wires: 12
gates: 58
depth: 46
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'C(CRZ)': 54, 'Adjoint(QFT

In [15]:
calculator.run()

RPN expression: 1 2 + 3 /


1+2
3/3
RPN calculator result=1
naive_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
pennylane_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
zx_specs:Graph(148 vertices, 183 edges)
degree distribution: 
1: 14
2: 50
3: 84



In [18]:
calculator.run()

RPN expression: 1 2 + 3 * 4 -


1+2
3*3
9-4
RPN calculator result=1
naive_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
pennylane_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
zx_specs:Graph(148 vertices, 183 edges)
degree distribution: 
1: 14
2: 50
3: 84

naive_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
pennylane_specs:wires: 7
gates: 16
depth: 8
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'CRZ': 12, 'Adjoint(QFT)': 1}
gate_sizes:
{2: 14, 3: 2}
zx_specs:Graph(148 vertices, 183 edges)
degree distribution: 
1: 14
2: 50
3: 84

naive_specs:wires: 12
gates: 58
depth: 46
shots: Shots(total=1)
gate_types:
{'BasisEmbedding': 2, 'QFT': 1, 'C(CRZ)': 54, 'Adjoint(QFT)': 1}
gate_

In [ ]:
calculator.run()

RPN expression: 3 4 * 5 * 6 *


3*4


/home/dan-seol/micromamba/envs/pennylane-codebook/lib/python3.12/site-packages/pennylane/gradients/gradient_transform.py:293: UserWarning: Attempted to compute the gradient of a tape with no trainable parameters. If this is unintended, please mark trainable parameters in accordance with the chosen auto differentiation framework, or via the 'tape.trainable_params' property.
  warnings.warn(_no_trainable_grad_warning)


12*5
60*6


# Limits

- Integer division -- problems with arbitrary number of qubits
- Making use of floating point arithmetic? https://msoeken.github.io/papers/2018_rc_2.pdf
- More operations -- exp, log, etc.
  